# Introduction

In this project, I'll build a model to predict the sentiment of movie reviews. This notebook contains the steps to clean the data for modelling and save it. 

In [1]:
import re

import pandas as pd
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import CountVectorizer

In [4]:
reviews = pd.read_csv('../data/IMDB_Dataset.csv')
reviews.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [7]:
reviews.iloc[0]['review']

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

In [8]:
# check for missing values

reviews.isnull().sum()

review       0
sentiment    0
dtype: int64

In [9]:
# check for duplicate values

reviews[reviews.duplicated()]

,review,sentiment
3537,Quite what the producers of this appalling ada...,negative
3769,My favourite police series of all time turns t...,positive
4391,"Beautiful film, pure Cassavetes style. Gena Ro...",positive
6352,If you liked the Grinch movie... go watch that...,negative
6479,I want very much to believe that the above quo...,negative
...,...,...
49912,This is an incredible piece of drama and power...,positive
49950,This was a very brief episode that appeared in...,negative
49984,Hello it is I Derrick Cannon and I welcome you...,negative
49986,This movie is a disgrace to the Major League F...,negative


In [10]:
reviews.shape

(50000, 2)

In [11]:
# Dropping duplicate rows to prevent data leakage

reviews = reviews.drop_duplicates()
reviews.shape

(49582, 2)

In [12]:
reviews.tail()

,review,sentiment
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative
49999,No one expects the Star Trek movies to be high...,negative


In [13]:
# Check for imbalance
reviews['sentiment'].value_counts()

positive    24884
negative    24698
Name: sentiment, dtype: int64

In [ ]:
reviews['review'].str.split(expand=True).head()

In [ ]:
word_counts = reviews['review'].str.split(expand=True).stack().value_counts()
word_counts.head(20)

In [14]:
#
stop_words = stopwords.words('english')
stop_words.remove('no')
stop_words.remove('but')
stop_words.remove('not')
stop_words = set(stop_words).union(['br'])

In [15]:

def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-Z0-9\s]|\[|\]' if not remove_digits else r'[^a-zA-Z\s]|\[|\]'
    text = re.sub(pattern, '', text)
    return text

In [16]:
reviews['review'] = reviews['review'].apply(remove_special_characters, remove_digits=True).str.lower()

In [17]:
# replace special characters and digits
reviews['review'] = reviews['review'].str.replace("(\\d|\\W)+",' ')

/tmp/ipykernel_6764/3139963510.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  reviews['review'] = reviews['review'].str.replace("(\\d|\\W)+",' ')


In [18]:

# lemmatize text

def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    return ' '.join([lemmatizer.lemmatize(w) for w in word_tokenize(text) if not w in stop_words])
reviews['review'] = reviews['review'].apply(lemmatize_text)

In [19]:
reviews.tail()

,review,sentiment
49995,thought movie right good job wasnt creative or...,positive
49996,bad plot bad dialogue bad acting idiotic direc...,negative
49997,catholic taught parochial elementary school nu...,negative
49998,im going disagree previous comment side maltin...,negative
49999,no one expects star trek movie high art but fa...,negative


In [20]:
reviews.iloc[0]['review']

'one reviewer mentioned watching oz episode youll hooked right exactly happened mebr first thing struck oz brutality unflinching scene violence set right word go trust not show faint hearted timid show pull no punch regard drug sex violence hardcore classic use wordbr called oz nickname given oswald maximum security state penitentary focus mainly emerald city experimental section prison cell glass front face inwards privacy not high agenda em city home manyaryans muslim gangsta latino christian italian irish moreso scuffle death stare dodgy dealing shady agreement never far awaybr would say main appeal show due fact go show wouldnt dare forget pretty picture painted mainstream audience forget charm forget romanceoz doesnt mess around first episode ever saw struck nasty surreal couldnt say ready but watched developed taste oz got accustomed high level graphic violence not violence but injustice crooked guard wholl sold nickel inmate wholl kill order get away well mannered middle class i

In [ ]:
reviews.to_csv('../data/reviews_cleaned.csv')